# Projeto Webscraping: Cálculo de Rentabilidade Diária e Anual de Fundos de Ações

**Autor:** Eduardo B. Scheffer

**Data:** 31/08/2020

**Objetivo:** Obter rentabilidade diária e YTD de uma lista de fundos de ações:

* Constellation Ações FIC FIA
* Sqadra Long-Only FIC FIA
* Bogari Value FIC FIA
* Núcleo FIC FIA
* Dynamo Cougar FIA

**Observação:** Abrir o arquivo "Projeto Brasil Capital 1.xlsx" antes de e após rodar este programa, para verificar sua funcionalidade

---
## Método I:
    Extração dos dados das cotas diretamente de fontes públicas

### Importar bibliotecas:

In [1]:
import pandas as pd
from datetime import date
from pandas import ExcelWriter
import requests
# !pip install openpyxl --> "descomente" essa linha caso não tenha o openpyxl instalado
from openpyxl import load_workbook

### Declarar CNPJ dos fundos:

In [2]:
fundos = {'Nome Fundo': ['Constellation Ações FIC FIA', 'Sqadra Long-Only FIC FIA', 'Bogari Value FIC FIA', 'Núcleo FIC FIA', 'Dynamo Cougar FIA'],
          'CNPJ': ['16.948.298/0001-04', '09.412.822/0001-54', '08.323.402/0001-39', '14.068.366/0001-07', '73.232.530/0001-39']         
         }
fundos = pd.DataFrame(fundos, columns = ['Nome Fundo', 'CNPJ'])
fundos['Rentabilidade diária'] = ""
fundos['Rentabilidade YTD'] = ""

### Obter dados através da CVM - Dados Abertos

In [3]:
def consulta_cvm(x):
    url = 'http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_{}.csv'.format(x)
    df = pd.read_csv(url, sep=";", usecols = ["CNPJ_FUNDO", "DT_COMPTC", "VL_QUOTA"])
    return df

### Obter cotas de fundos da CVM referentes do ano atual (YTD)

In [4]:

request = requests.get('http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_{202009.csv')
if request.status_code == 200:
    print('Web site exists')
else:
    print('Web site does not exist') 

Web site does not exist


In [8]:
year = date.today().year # Ano atual
month = date.today().month # Mês atual
day = date.today().day # Dia atual

if day ==1: month = month - 1

for m in range (1,month+1):
    
    yyyy = str(year)
    if m < 10: mm = '0'+str(m)
    else: mm = str(m)
    
    data = yyyy+mm # Ano e mes para substituir no url
    
    request = requests.get(url = 'http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_{}.csv'.format(data))
    if request.status_code == 200: # Checa se site existe (ex.: no início do mês, pode ser que a CVM demore alguns dias para fazer o upload do primeiro arquivo)
        
        cvm_m = consulta_cvm(data)
        if m == 1:
            firstD = cvm_m['DT_COMPTC'].iloc[0] # Verifica qual é o primeiro dia do ano
            lastD = cvm_m['DT_COMPTC'].iloc[len(cvm_m)-1] # Verifica qual é o primeiro dia do ano
            cvm = cvm_m
        else:
            cvm = pd.concat([cvm,cvm_m])

### Obter as cotas dos fundos selecionados

In [9]:
quotas_fundos = {} #array de dataframes
for x in range(0, len(fundos)):
    cnpj = fundos['CNPJ'].iloc[x]
    quotas_fundos[x] = pd.DataFrame(cvm.loc[cvm['CNPJ_FUNDO'] == cnpj]) # Copia nova df para cada fundo com as informações da CVM relativos ao seu CNPJ
    quotas_fundos[x].drop('CNPJ_FUNDO', axis=1, inplace=True) # Deleta coluna do CNPJ para "limpar" a df (já que todas as linhas serão iguais)
    quotas_fundos[x].reset_index(drop=True,inplace=True) # Renumera os indices, para começar do 0

    

### Calcular as variações percentuais diárias da cota de cada fundo

In [10]:
for x in range(0, len(quotas_fundos)):
    quotas_fundos[x].insert(len(quotas_fundos[x].columns),'VAR_DIAR','') # Insere nova coluna para a rentabilidade diária
    quotas_fundos[x].insert(len(quotas_fundos[x].columns),'VAR_CUMU','') # Insere nova coluna para a rentabilidade cumulativa
    i=0
    while i+1 < len(quotas_fundos[x]):
        if i == 0:
            quotas_fundos[x].at[i,'VAR_DIAR'] = 0 # Normaliza a primeira data com rentabilidade diária 0
            quotas_fundos[x].at[i,'VAR_CUMU'] = 0 # Normaliza a primeira data com rentabilidade acumulada 0
        i = i+1;
        rd1 = float(str(quotas_fundos[x].at[i-1,'VL_QUOTA'])) # Valor da cota do dia anterios
        rd2 = float(str(quotas_fundos[x].at[i,'VL_QUOTA'])) # Valor da cota do dia
        vd = rd2 / rd1 - 1
        quotas_fundos[x].at[i,'VAR_DIAR'] = vd # Calcula a variação do valor da cota (rentabilidade)
        quotas_fundos[x].at[i,'VAR_CUMU'] = (1 + vd) * (1 + quotas_fundos[x].at[i-1,'VAR_CUMU']) -1 # Calcula a variação do valor da cota (rentabilidade)
        

### Calcular a rentabilidade diária mais recente e a rentabilidade do ano (YTD) para os fundos selecionados

In [11]:
for x in range(0, len(fundos)):
    l = len(quotas_fundos[x])
    fundos['Rentabilidade diária'].iloc[x] =  quotas_fundos[x].at[l-1,'VAR_DIAR']
    fundos['Rentabilidade YTD'].iloc[x] =  quotas_fundos[x].at[l-1,'VAR_CUMU']

### Print do Resultado:

In [12]:
fundos

,Nome Fundo,CNPJ,Rentabilidade diária,Rentabilidade YTD
0,Constellation Ações FIC FIA,16.948.298/0001-04,-0.0215065,0.0345625
1,Sqadra Long-Only FIC FIA,09.412.822/0001-54,-0.0237993,-0.0757578
2,Bogari Value FIC FIA,08.323.402/0001-39,-0.00690678,-0.0194474
3,Núcleo FIC FIA,14.068.366/0001-07,-0.0184367,0.0447758
4,Dynamo Cougar FIA,73.232.530/0001-39,-0.0212341,0.0532115


### Exportar para o Excel em nova aba ("Input")

In [13]:
wb = load_workbook("Projeto Brasil Capital 1.xlsx") #Abre a planilha a ser preenchida

writer = pd.ExcelWriter("Projeto Brasil Capital 1.xlsx")
writer.book = wb
writer.sheets = dict((ws.title, ws) for ws in wb.worksheets) # Possibilita escrever em aba já existente em caso de atualização dos dados

fundos.to_excel(writer, sheet_name='Input', index = False) # Copia informações do fundo na aba "Input"
writer.save()

### Exportar para o Excel em na aba "Projeto_2020"

In [14]:
ws = wb['Projeto_2020']
for x in range(0, len(fundos)):
    ws.cell(row = 12+x, column=4).value = fundos['Rentabilidade diária'].iloc[x]
    ws.cell(row = 12+x, column=5).value = fundos['Rentabilidade YTD'].iloc[x]
writer.save()